# Preparação Ontologia TCE/RN

In [1]:
import os
import json
import pymssql

import pandas as pd
import numpy as np


from pprint import pprint
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from sqlalchemy import (
    create_engine, Column, Integer, String, Float, Date, Boolean, Text, JSON
)
from sqlalchemy.orm import declarative_base, sessionmaker

from tools.prompt import generate_few_shot_ner_prompts, extract_obrigacao, extract_recomendacao, get_pessoas_str
from tools.schema import (
    NERDecisao,
    Obrigacao,
    Recomendacao,
    BeneficioBase
)

from tools.models import (
    Base, ObrigacaoORM, RecomendacaoORM
)

from datetime import timedelta


load_dotenv()


gpt4turbo = AzureChatOpenAI(
    deployment_name="gpt-4-turbo",
    model_name="gpt-4",
)

'''
gpt4turbo = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.0
)
'''

extractor_gpt4turbo = gpt4turbo.with_structured_output(NERDecisao, include_raw=False, method="json_schema")
extractor_obrigacao_gpt4turbo = gpt4turbo.with_structured_output(
    Obrigacao, include_raw=False, method="json_schema")
extractor_recomendacao_gpt4turbo = gpt4turbo.with_structured_output(
    Recomendacao, include_raw=False, method="json_schema")
extractor_beneficio_gpt4turbo = gpt4turbo.with_structured_output(
    BeneficioBase, include_raw=False, method="json_schema")


def get_connection(db: str = 'processo'):
    server = os.getenv("SQL_SERVER_HOST")
    user = os.getenv("SQL_SERVER_USER")
    password = os.getenv("SQL_SERVER_PASS")
    port = os.getenv("SQL_SERVER_PORT", "1433")  # default MSSQL port
    database = db

    return pymssql.connect(server, user, password, database, port)

conn_bddip = get_connection(db="BdDIP")
conn = get_connection()


c:\Users\05911205424\Documents\Dev\cgad\venv\Lib\site-packages\langchain_openai\chat_models\base.py:1927: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [2]:
sql_obrigacoes = """

SELECT ai.setor, ai.ementa, ai.assunto, ai.Relatorio, ai.FundamentacaoVoto, ai.decisao, 
ai.conclusao, ai.numero_sessao, ai.ano_sessao,
ai.texto_acordao, ai.DataSessao, CONCAT(p.numero_processo, '/', p.ano_processo) as processo,
o.nome as orgao_responsavel, 
o.IdOrgao as id_orgao_responsavel,
gp.Nome as nome_responsavel,
gp.Documento as documento_responsavel,
gp.TipoPessoa  as tipo_responsavel,
gp.IdPessoa as id_pessoa
,
ob.*
FROM BdDIP.dbo.Obrigacao ob
INNER JOIN processo.dbo.Processos p
ON ob.IdProcesso = p.IdProcesso 
INNER JOIN processo.dbo.vw_ia_votos_acordaos_decisoes ai
ON ai.IdVotoPauta = ob.IdVotoPauta and ai.IdComposicaoPauta = ob.IdComposicaoPauta
INNER JOIN processo.dbo.Orgaos o ON p.IdOrgaoEnvolvido = o.IdOrgao 
INNER JOIN processo.dbo.Pro_ProcessosResponsavelDespesa pprd ON pprd.IdProcesso = p.IdProcesso 
INNER JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = pprd.IdPessoa 

"""
df_obrigacoes = pd.read_sql(sql_obrigacoes, conn_bddip)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_1652\2557224626.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_obrigacoes = pd.read_sql(sql_obrigacoes, conn_bddip)


In [3]:
import re

def slugify(s):
    s = s.lower()
    s = re.sub(r'[^\w\s-]', '', s)
    s = re.sub(r'[\s]+', '_', s)
    return s

df_obrigacoes.columns = [slugify(col) for col in df_obrigacoes.columns]

In [4]:
df_obrigacoes.columns

Index(['setor', 'ementa', 'assunto', 'relatorio', 'fundamentacaovoto',
       'decisao', 'conclusao', 'numero_sessao', 'ano_sessao', 'texto_acordao',
       'datasessao', 'processo', 'orgao_responsavel', 'id_orgao_responsavel',
       'nome_responsavel', 'documento_responsavel', 'tipo_responsavel',
       'id_pessoa', 'idobrigacao', 'idprocesso', 'idcomposicaopauta',
       'idvotopauta', 'descricaoobrigacao', 'defazer', 'prazo',
       'datacumprimento', 'orgaoresponsavel', 'idorgaoresponsavel',
       'temmultacominatoria', 'nomeresponsavelmultacominatoria',
       'documentoresponsavelmultacominatoria', 'idpessoamultacominatoria',
       'valormultacominatoria', 'periodomultacominatoria',
       'emultacominatoriasolidaria', 'solidariosmultacominatoria',
       'cancelado'],
      dtype='object')

In [5]:
group_cols = [
        'idcomposicaopauta', 'ementa', 'assunto',
        'idvotopauta', 'numero_sessao', 'ano_sessao', 'datasessao', 'relatorio',
        'fundamentacaovoto', 'conclusao', 'texto_acordao', 'orgao_responsavel', 'id_orgao_responsavel', 'processo', 'descricaoobrigacao'
]
person_cols = ['nome_responsavel', 'documento_responsavel', 'tipo_responsavel', 'id_pessoa']

df_obrigacoes = df_obrigacoes.groupby(group_cols, dropna=False).apply(
    lambda x: pd.Series({'responsaveis': x[person_cols].apply(
        lambda y: y.dropna().to_dict(), axis=1
    ).tolist()})
).reset_index()

C:\Users\05911205424\AppData\Local\Temp\ipykernel_1652\1229705928.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_obrigacoes = df_obrigacoes.groupby(group_cols, dropna=False).apply(


In [6]:
df_obrigacoes

,idcomposicaopauta,ementa,assunto,idvotopauta,numero_sessao,ano_sessao,datasessao,relatorio,fundamentacaovoto,conclusao,texto_acordao,orgao_responsavel,id_orgao_responsavel,processo,descricaoobrigacao,responsaveis
0,75885,EMENTA: VOTO-VISTA. CONCESSÃO DE MEDIDA CAUTEL...,AUDITORIA DE REGULARIDADE NO DEPARTAMENTO ESTA...,16072,00024,2018,2018-07-03 00:00:00,Versam os autos sobre auditoria promovida por ...,"Conforme relatado, nesse juízo de cognição sum...","Diante de todo o exposto, com fulcro no art. 1...","Vistos, relatados e discutidos estes autos da ...",DEPARTAMENTO ESTADUAL DE TRÂNSITO,202,002283/2017,"Apresentar, no prazo de 30 dias, os relatórios...",[{'nome_responsavel': 'Departamento Estadual d...
1,78264,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. APOSEN...,APRECIAÇÃO DE CONCESSÃO DE APOSENTADORIA ...,17226,00098,2018,2018-12-18 00:00:00,Trata-se da análise do ato de aposentadoria co...,Pelo que se constata nos documentos acostados ...,"Do exposto, em consonância com a análise do Co...",DECIDEM os Conselheiros do Tribunal de Contas ...,INSTITUTO DE PREVIDÊNCIA SOCIAL DE SÃO GONÇALO...,1189,000201/2017,Adotar medidas regularizadoras cabíveis.,[{'nome_responsavel': 'Célia Maria da Cunha Br...
2,96707,PROCESSO Nº: 4324/2017 – TC\r\nINTERESSADO (A)...,APRECIAÇÃO DE CONCESSÃO DE APOSENTADORIA ...,25256,00032,2021,2021-05-13 00:00:00,I - RELATÓRIO\r\n\r\nVersam os autos acerca da...,II – FUNDAMENTAÇÃO\r\n\r\nA matéria sob exame ...,"III – CONCLUSÃO\r\n\r\nAnte o exposto, em harm...",DECIDEM os Conselheiros do Tribunal de Contas ...,"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",513,004324/2017,Expedição de determinação ao Instituto de Prev...,[{'nome_responsavel': 'ANTONIA MARIA DE SOUSA ...
3,96707,PROCESSO Nº: 4324/2017 – TC\r\nINTERESSADO (A)...,APRECIAÇÃO DE CONCESSÃO DE APOSENTADORIA ...,25256,00032,2021,2021-05-13 00:00:00,I - RELATÓRIO\r\n\r\nVersam os autos acerca da...,II – FUNDAMENTAÇÃO\r\n\r\nA matéria sob exame ...,"III – CONCLUSÃO\r\n\r\nAnte o exposto, em harm...",DECIDEM os Conselheiros do Tribunal de Contas ...,"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",513,004324/2017,"O gestor responsável deverá, no prazo de 05 (c...",[{'nome_responsavel': 'ANTONIA MARIA DE SOUSA ...
4,106724,EMENTA: CONSTITUCIONAL. ADMINISTRATIVO. PREVID...,APOSENTADORIA POR TEMPO DE CONTRIBUIÇÃO ...,32982,00003,2023,2023-01-31 00:00:00,"O presente processo, autuado neste Tribunal em...","Ab initio, considerando que ainda não se opero...","ISTO POSTO, acolhendo a Informação do Corpo Té...",DECIDEM os Conselheiros do Tribunal de Contas ...,INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO RN,308,101837/2018,"Corrigir o ato aposentador, a apostila de cálc...",[{'nome_responsavel': 'FERNANDO ANTONIO MARQUE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,123503,FISCALIZAÇÃO DE ATO DE PESSOAL SUJEITO A REGIS...,APRECIAÇÃO DA APOSENTADORIA CONCEDIDA AO(À) SE...,46199,0020v,2025,2025-10-20 10:00:00,Trata-se de processo referente à concessão de ...,A concessão de aposentadorias no serviço públi...,"Em razão do exposto, acatando a manifestação d...",\r\nDECIDEM os Conselheiros do Tribunal de Con...,REGIME PRÓPRIO DE PREVIDÊNCIA SOCIAL DO MUNICÍ...,1223,100859/2020,"Intimar a Prefeitura Municipal de Itaú, por me...","[{'nome_responsavel': 'MARIA ALVES MAIA', 'doc..."
1069,123504,FISCALIZAÇÃO DE ATO DE PESSOAL SUJEITO A REGIS...,APRECIAÇÃO DA APOSENTADORIA CONCEDIDA AO(À) SE...,46196,0020v,2025,2025-10-20 10:00:00,Trata-se de processo de aposentadoria de LUCIA...,A concessão de aposentadorias no serviço públi...,"Em razão do exposto, acatando a manifestação d...",\r\nDECIDEM os Conselheiros do Tribunal de Con...,REGIME PRÓPRIO DE PREVIDÊNCIA SOCIAL DO MUNICÍ...,1223,100004/2021,Determinação ao Órgão Previdenciário responsáv...,[{'nome_responsavel': 'LUCIA MARIA DE SOUZA MA...
1070,123598,REPRESENTAÇÃO. ACUMULAÇÃO DE CARGOS PÚBLICOS. ...,ACUMULAÇÃO DE CARGOS PÚBLICOS (ID15 - PFA 20/2...,46237,0021V,2025,2025-11-03 10:

In [7]:
row = df_obrigacoes[df_obrigacoes['processo'] == '007938/2018'].iloc[1]

In [8]:
df_obrigacoes[df_obrigacoes['processo'] == '007938/2018']

,idcomposicaopauta,ementa,assunto,idvotopauta,numero_sessao,ano_sessao,datasessao,relatorio,fundamentacaovoto,conclusao,texto_acordao,orgao_responsavel,id_orgao_responsavel,processo,descricaoobrigacao,responsaveis
434,113221,EMENTA: REPRESENTAÇÃO. ACUMULAÇÕES DE VÍNCULOS...,REPRESENTAÇÃO -ACUMULAÇÃO DE CARGOS PÚBLICOS (...,38157,00001,2024,2024-01-25,Tratam os autos de representação ofertada pela...,"Inicialmente, destaco que a presente Represent...","Ante o exposto, concordando, parcialmente, com...","Vistos, relatados e discutidos estes autos, co...",PREFEITURA MUNICIPAL DE TIBAU DO SUL,494,007938/2018,Que o atual Prefeito do Município de Tibua do ...,[{'nome_responsavel': 'antonio modesto rodrigu...
435,113221,EMENTA: REPRESENTAÇÃO. ACUMULAÇÕES DE VÍNCULOS...,REPRESENTAÇÃO -ACUMULAÇÃO DE CARGOS PÚBLICOS (...,38157,00001,2024,2024-01-25,Tratam os autos de representação ofertada pela...,"Inicialmente, destaco que a presente Represent...","Ante o exposto, concordando, parcialmente, com...","Vistos, relatados e discutidos estes autos, co...",PREFEITURA MUNICIPAL DE TIBAU DO SUL,494,007938/2018,Que seja fixado o prazo de 120 (cento e vinte)...,[{'nome_responsavel': 'antonio modesto rodrigu...


In [104]:
base_prompt = f"""
EMENTA: {row['ementa']}
ASSUNTO: {row['assunto']}
RELATÓRIO: {row['relatorio']}
FUNDAMENTAÇÃO DO VOTO: {row['fundamentacaovoto']}
ACÓRDÃO: {row['texto_acordao']}
"""

In [105]:
prompt = f"""
    Você é um Auditor de Controle Externo do TCE/RN. Sua tarefa é analisar o voto e extrair a obrigação imposta, preenchendo os campos do objeto Obrigacao.

    Com base no seguinte contexto:
    {base_prompt}

    Data da Sessão: {row['datasessao'].strftime('%d/%m/%Y')}
    Obrigação detectada: {row['descricaoobrigacao']}
    Texto do Acordão: {row['texto_acordao']}
    Órgão Responsável: {row['orgao_responsavel']}
    Pessoas Responsáveis: {get_pessoas_str(row['responsaveis'])}

    Dado esse contexto, preencha os campos da seguinte forma:
    - descricao_obrigacao: Descrição da obrigação imposta. Se a descrição for ampla e abstrata, descreva melhor o conteúdo da obrigação com base no contexto fornecido.
    - tipo: Tipo da obrigação (fazer/não fazer).
    - prazo: Prazo estipulado para cumprimento. Extraia o texto indicando o prazo, se houver. Exemplo: "90 dias".
    - data_cumprimento: Extraia do prazo do acórdão como data de início e faça o cálculo da data de cumprimento. Exemplo: 2025-09-13
    - orgao_responsavel: Órgão responsável pelo cumprimento da obrigação. Pessoa jurídica.
    - tem_multa_cominatoria: Indique se há multa cominatória associada à obrigação.
    - nome_responsavel_multa_cominatoria: Nome do responsável pela obrigação, se houver multa cominatória. Pessoa física responsável.
    - documento_responsavel_multa_cominatoria: Documento do responsável pela obrigação, se houver multa cominatória.
    - valor_multa_cominatoria: Se houver multa cominatória, preencha o valor.
    - periodo_multa_cominatoria: Período da multa cominatória, se houver.
    - e_multa_cominatoria_solidaria: Indique se a multa cominatória é solidária.
    - solidarios_multa_cominatoria: Lista de responsáveis solidários da multa cominatória.

    Use somente as informações do texto do acórdão e dos dados fornecidos. Não inclua informações adicionais ou suposições.
    Se o órgão responsável não estiver disponível, preencha o campo orgão_responsavel com "Desconhecido".
    """

In [106]:
result = extractor_obrigacao_gpt4turbo.invoke(prompt)

In [107]:
result

Obrigacao(descricao_obrigacao='A Prefeitura Municipal de Tibau do Sul deve instaurar processos administrativos individuais para apurar a legalidade da acumulação de vínculos públicos dos servidores especificados, assegurando contraditório, ampla defesa e devido processo legal, e concluir esses processos em 120 dias úteis, além de reportar ao TCE em até 5 dias úteis após o prazo.', de_fazer=True, prazo='120 dias úteis para conclusão dos processos administrativos, com 5 dias úteis adicionais para comunicação ao TCE.', data_cumprimento=datetime.date(2024, 7, 12), orgao_responsavel='PREFEITURA MUNICIPAL DE TIBAU DO SUL', tem_multa_cominatoria=True, nome_responsavel_multa_cominatoria='Atual gestor da Prefeitura Municipal de Tibau do Sul', documento_responsavel_multa_cominatoria='08168775000182', valor_multa_cominatoria=500.0, periodo_multa_cominatoria='diário', e_multa_cominatoria_solidaria=False, solidarios_multa_cominatoria=None)

In [101]:
print(row['texto_acordao'])

Vistos, relatados e discutidos estes autos, concordando, parcialmente, com o corpo técnico e com o Ministério Público de Contas, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro Relator, julgar  pelo conhecimento da representação em tela, mediante o julgamento pela IRREGULARIDADE do mérito da matéria analisada, com fulcro no art. 75, inciso II, da Lei Complementar Estadual nº 464/2012, determinando-se o seguinte:
a) Que o atual Prefeito do Município de Tibua do Sul instaure processos administrativos disciplinares individuais, que tenham por escopo a apuração da legalidade da acumulação de vínculos públicos dos servidores Edna Mércia de Lima Marinho, Daniel Frederico Fagundes de Lima Andrade, Micarla Oliveira da Silva, Fabio Antonio de Oliveira, Cleidson Costa de Lima e Wellington de Oliveira Junior, sendo-lhes assegurado o contraditório, a ampla defesa e o devido processo legal;
b) Que seja fixado o prazo de 120 (cento em vinte) dias úteis para a conclusão dos alu

In [9]:
base_prompt = f"""
EMENTA: {row['ementa']}
ASSUNTO: {row['assunto']}
RELATÓRIO: {row['relatorio']}
FUNDAMENTAÇÃO DO VOTO: {row['fundamentacaovoto']}
ACÓRDÃO: {row['texto_acordao']}
"""

prompt_beneficio = f"""
Você é um Auditor de Controle Externo do TCE/RN. Sua tarefa é analisar o voto e o acórdão
e extrair o BENEFÍCIO gerado pela atuação do Tribunal, preenchendo os campos do objeto Beneficio.

Considere os conceitos do Manual de Quantificação de Benefícios do TCE/RN:
- Tipos de benefício (campo tipo_beneficio):
  - SANCAO: sanções aplicadas pelo Tribunal (multa, inabilitação, inidoneidade).
  - RESTITUICAO: restituição de recursos ao Erário (débitos imputados, devoluções, restituição voluntária).
  - CORRECAO: correção de irregularidades ou impropriedades (glosa, redução contratual, revisão tarifária etc.).
  - PESSOAL_PREVIDENCIA: benefícios financeiros na área de pessoal e previdência (suspensão de pagamentos indevidos, devolução de vantagens etc.).
  - INCREMENTO_GESTAO: incremento de economia, eficiência, eficácia ou efetividade de órgão/programa de governo.
  - OUTROS: outros benefícios não enquadrados nas categorias acima.

- Subtipos de benefício (campo subtipo_beneficio), quando aplicável:
  - MULTA, INABILITACAO, INIDONEIDADE
  - DEBITO_IMPUTADO, RESTITUICAO_ADMINISTRACAO, RESTITUICAO_VOLUNTARIA
  - REDUCAO_TARIFA_REVISAO, GLOSA_DESPESA, REDUCAO_VALOR_CONTRATUAL,
    COMPENSACAO_FINANCEIRA, EXECUCAO_GARANTIA, MULTA_CONTRATUAL,
    CORRECAO_VICIOS, INCOMPATIBILIDADE_OBJETO, REDUCAO_PRECO_MAXIMO,
    APERFEICOAMENTO_CUSTOS, ELEVACAO_PRECO_MINIMO_OUTORGA, REDUCAO_TARIFA_LICITACAO
  - DEVOLUCAO_VANTAGEM, SUSPENSAO_VANTAGEM_TEMP,
    SUSPENSAO_VANTAGEM_INDETERMINADO, SUSPENSAO_PREVIDENCIA,
    SUSPENSAO_TEMPORARIO, SUSPENSAO_EFETIVO
  - ELIMINACAO_DESPERDICIOS, ELEVACAO_RECEITA,
    MELHORIA_ATENDIMENTO, MELHORIA_RISCOS_CONTROLES, OUTRO_INCREMENTO
  - OUTRO (quando nenhum subtipo anterior se encaixar claramente)

- Característica do benefício (campo caracteristica):
  - QUANTITATIVO_FINANCEIRO: quando puder ser expresso em moeda (R$).
  - QUANTITATIVO_NAO_FINANCEIRO: quando a medida é numérica, mas não monetária
    (número de beneficiários, hectares preservados, quantidade de equipamentos etc.).
  - QUALITATIVO: quando não for possível ou adequado quantificar (ex.: melhoria normativa,
    aumento da transparência, fortalecimento de controles, redução de sentimento de impunidade).

Assuma que, no contexto do acórdão colegiado, o benefício está no estágio de:
- estagio = "POTENCIAL"
- ocorrencia = "Decisão"

Use o seguinte contexto:

{base_prompt}

Dados adicionais:
- Data da Sessão: {row['datasessao'].strftime('%d/%m/%Y')}
- Benefício detectado (rótulo prévio, se houver): {row.get('descricaobeneficio', '')}
- Texto do Acórdão: {row['texto_acordao']}
- Órgão Responsável (decisor/beneficiário): {row.get('orgao_responsavel', '')}
- Pessoas Responsáveis, se relacionadas ao benefício: {get_pessoas_str(row['responsaveis'])}

Com base nesse contexto, preencha os campos do objeto Beneficio da seguinte forma:

- tipo_beneficio:
  Escolha UMA das opções: ["SANCAO", "RESTITUICAO", "CORRECAO", "PESSOAL_PREVIDENCIA",
  "INCREMENTO_GESTAO", "OUTROS"].
  Baseie-se no efeito prático da decisão:
  - Se há multa, inabilitação ou inidoneidade aplicada pelo Tribunal → "SANCAO".
  - Se há devolução/ressarcimento de valores ao Erário → "RESTITUICAO".
  - Se há correção de contrato, tarifa, glosa, compensação, execução de garantia etc. → "CORRECAO".
  - Se trata de área de pessoal/previdência (aposentadorias, pensões, folha, vantagens indevidas) → "PESSOAL_PREVIDENCIA".
  - Se melhora gestão, eficiência, eficácia, efetividade ou receita de órgão/programa → "INCREMENTO_GESTAO".
  - Caso contrário, use "OUTROS".

- subtipo_beneficio:
  Escolha UMA das opções listadas acima para subtipo.
  Se nenhum subtipo encaixar claramente, use "OUTRO".

- caracteristica:
  Escolha UMA das opções: ["QUANTITATIVO_FINANCEIRO", "QUANTITATIVO_NAO_FINANCEIRO", "QUALITATIVO"].
  - Se o benefício puder ser expresso em R$ com base no texto → "QUANTITATIVO_FINANCEIRO".
  - Se há medida numérica não financeira claramente extraível (ex.: hectares, número de beneficiários) → "QUANTITATIVO_NAO_FINANCEIRO".
  - Se não for possível ou adequado quantificar com segurança → "QUALITATIVO".

- valor:
  - Se caracteristica = "QUANTITATIVO_FINANCEIRO", preencha com o valor monetário do benefício (somente números, em reais).
    Exemplo: 2000000.00
  - Se não for possível determinar valor exato, mas o benefício é claramente financeiro, estime um valor conservador somente
    se o texto permitir. Caso contrário, deixe nulo.
  - Se caracteristica ≠ "QUANTITATIVO_FINANCEIRO", deixe nulo.

- quantidade:
  - Se caracteristica = "QUANTITATIVO_NAO_FINANCEIRO", preencha com a quantidade estimada (por exemplo, número de unidades,
    beneficiários, hectares etc.), se houver base no texto.
  - Caso contrário, deixe nulo.

- descricao:
  Descreva, em português claro, qual é o benefício gerado pela decisão.
  A descrição deve responder: "Que ganho concreto ou potencial essa decisão produz para o Erário, para a Administração
  Pública ou para a sociedade?".
  Se a descrição do benefício for muito ampla ou vaga no texto, refine e explicite melhor com base no contexto fornecido.

- memoria_calculo:
  Explique, em texto, como chegou ao valor ou quantidade (quando houver), mencionando:
  - a base de cálculo (ex.: valor do contrato, número de parcelas, período em anos/meses, remuneração mensal),
  - a fórmula em linguagem natural (ex.: "50 parcelas restantes × R$ 1.000.000,00"),
  - eventual prazo considerado (ex.: "próximos 12 meses", "até a expectativa de vida do beneficiário").
  Se o benefício for qualitativo e não houver cálculo, escreva uma breve justificativa textual do impacto qualitativo.

- estagio:
  Use sempre "POTENCIAL", pois estamos analisando a decisão do Tribunal.

- ocorrencia:
  Use "Decisão", pois o benefício está sendo identificado na fase decisória do acórdão.

- area_tematica:
  Se possível, indique uma área temática geral (ex.: "Pessoal e Previdência", "Obras e Engenharia", "Licitações e Contratos",
  "Receita e Arrecadação", "Meio Ambiente"). Caso não seja possível inferir, use "Geral".

Regras importantes:
- Use SOMENTE as informações do texto do acórdão e dos dados fornecidos.
- Não invente valores monetários ou quantidades que não tenham respaldo mínimo no texto.
- Se algum campo não puder ser preenchido com segurança, deixe-o nulo.
- A saída deve ser APENAS o objeto Beneficio em formato JSON, respeitando exatamente os nomes dos campos:
  {{
    "tipo_beneficio": "...",
    "subtipo_beneficio": "...",
    "caracteristica": "...",
    "valor": ...,
    "quantidade": ...,
    "descricao": "...",
    "memoria_calculo": "...",
    "estagio": "POTENCIAL",
    "ocorrencia": "Decisão",
    "area_tematica": "..."
  }}
"""
result_beneficio = extractor_beneficio_gpt4turbo.invoke(prompt_beneficio)

In [10]:
result_beneficio

BeneficioBase(IdProcesso=0, IdPFA=None, NumeroProcesso=None, DimensaoFiscalizacao=None, InstrumentoFiscalizacao=None, NumeroAcordao=None, Encaminhamento=None, TipoBeneficio=<TipoBeneficio.PESSOAL_PREVIDENCIA: 'PESSOAL_PREVIDENCIA'>, SubtipoBeneficio='SUSPENSAO_VANTAGEM_INDETERMINADO', AreaTematica=None, Estagio=<EstagioBeneficio.POTENCIAL: 'POTENCIAL'>, Ocorrencia='Decisão', Caracteristica=<CaracteristicaBeneficio.QUANTITATIVO_NAO_FINANCEIRO: 'QUANTITATIVO_NAO_FINANCEIRO'>, Valor=None, Quantidade=6.0, Descricao='Suspensão de vínculos públicos indevidos de servidores na Prefeitura Municipal de Tibau do Sul, garantindo a legalidade da acumulação de cargos e contribuindo para a regularidade nas despesas com pessoal.', MemoriaCalculo='A quantidade é baseada nos 6 casos remanescentes de acumulação de cargos públicos, conforme identificado na decisão.', DataRegistro=None)

In [13]:
print(row['texto_acordao'])

Vistos, relatados e discutidos estes autos, concordando, parcialmente, com o corpo técnico e com o Ministério Público de Contas, ACORDAM os Conselheiros, nos termos do voto proferido pelo Conselheiro Relator, julgar  pelo conhecimento da representação em tela, mediante o julgamento pela IRREGULARIDADE do mérito da matéria analisada, com fulcro no art. 75, inciso II, da Lei Complementar Estadual nº 464/2012, determinando-se o seguinte:
a) Que o atual Prefeito do Município de Tibua do Sul instaure processos administrativos disciplinares individuais, que tenham por escopo a apuração da legalidade da acumulação de vínculos públicos dos servidores Edna Mércia de Lima Marinho, Daniel Frederico Fagundes de Lima Andrade, Micarla Oliveira da Silva, Fabio Antonio de Oliveira, Cleidson Costa de Lima e Wellington de Oliveira Junior, sendo-lhes assegurado o contraditório, a ampla defesa e o devido processo legal;
b) Que seja fixado o prazo de 120 (cento em vinte) dias úteis para a conclusão dos alu